In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use("fivethirtyeight")
from time import time

In [ ]:
mushroomdata=pd.read_csv("/kaggle/input/mushroom-classification/mushrooms.csv")
mushroomdata.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
cols=mushroomdata.columns

In [ ]:
encoding=LabelEncoder()

In [ ]:
for i in cols:
    mushroomdata[i]=encoding.fit_transform(mushroomdata[i])

In [ ]:
mushroomdata.head()

In [ ]:
mushroomdata.corr()

In [ ]:
mushroomdata.describe().transpose()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:

x=mushroomdata.drop("class",axis=1).values
y=mushroomdata["class"].values
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=12)

In [ ]:

def modelcreating(optimizer="adam",loss="binary_crossentropy"):
    model=Sequential()
    model.add(Dense(22,input_dim=22,activation="relu"))
    model.add(Dense(22,activation="relu"))
    model.add(Dense(11,activation="relu"))
    model.add(Dense(5,activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss=loss,optimizer=optimizer,metrics=["accuracy"])
    return model
model=modelcreating()

In [ ]:
earlystopping=EarlyStopping(monitor="val_loss",mode="min",patience=25)

In [ ]:

tensorboard=TensorBoard(log_dir="logs\{}".format(time()),
                        histogram_freq=1,
                       write_graph=True,)

model.fit(x=x_train,y=y_train,
          validation_data=(x_test,y_test),
         epochs=100,batch_size=30,verbose=1,
         callbacks=[tensorboard])

In [ ]:
loss=pd.DataFrame(model.history.history)
loss.head()

In [ ]:
plt.figure(figsize=(10,6))
loss[["loss","val_loss"]].plot()

In [ ]:
plt.figure(figsize=(10,6))
loss[["accuracy","val_accuracy"]].plot()

In [ ]:
y_pred=model.predict_classes(x_test)


In [ ]:

# analysing the results
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:

scores=model.evaluate(x_test,y_test)
print(model.metrics_names[1],scores[1]*100)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs